In [156]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [39]:
lekagul_df = pd.read_csv('hw03-data/Traffic Data/Lekagul Sensor Data.csv')
lekagul_df['Timestamp'] = pd.to_datetime(lekagul_df['Timestamp'])

In [105]:
# [car_id, car_type, entry_timestamp, exit_timestamp, length_of_stay]
def compute_length_of_stay_row(car_id):
    relevant_data = lekagul_df[lekagul_df['car-id'] == car_id]
    length_of_stay = (relevant_data['Timestamp'].max() - relevant_data['Timestamp'].min()).total_seconds()/60/60
    car_type = relevant_data.iloc[0]['car-type']
    return [car_id, car_type, relevant_data['Timestamp'].min(), relevant_data['Timestamp'].max(),length_of_stay]

In [106]:
# create list: [car id, entry timestamp, length of stay]
unique_ids = lekagul_df['car-id'].unique()
length_of_stay_data = []
for unique_id in unique_ids:
    length_of_stay_data.append(compute_length_of_stay_row(unique_id))

In [112]:
# create dataframe from list
length_of_stay_df = pd.DataFrame(length_of_stay_data, columns=['car-id', 'car-type', 'entry-timestamp', 'exit-timestamp', 'length-of-stay-in-hours'])

In [162]:
# https://martinxpn.medium.com/creating-beautiful-data-visualizations-with-plotly-and-dash-96-100-days-of-python-bbfdfd333a3b
app = Dash(__name__)
app.layout = html.Div([
    dcc.Input(
            id="max_y",
            type='number',
            placeholder="max y value",
        ),
    dcc.Graph(id='my_graph', figure=figure)
])

@app.callback(
    Output(component_id='my_graph', component_property='figure'),
    Input(component_id='max_y', component_property='value')
)
def update_graph(max_y):

    # filtered_df = length_of_stay_df[length_of_stay_df['length-of-stay-in-hours'] < max_y]
    
    fig = px.scatter(length_of_stay_df, 
                 x='entry-timestamp', 
                 y='length-of-stay-in-hours', 
                 color='car-type', 
                 hover_data='car-id')
    
    # create the figure
    figure = go.Figure(data=fig)

    # Add the range slider
    figure.update_layout(
        title="Arrival Time vs Length of Stay in Hours",
        yaxis=dict(
            title='Length of stay (hours)',
        ),
        xaxis=dict(
            title='Entry Timestamp',
            rangeslider=dict(
                visible=True,
                thickness=0.025,
                bgcolor='#AAAAAA'
            ), # Enables the range slider
            rangeselector=dict(
                buttons=list([
                    dict(step="all"),
                ]),
            )
        ),
        yaxis_range=[0,max_y]
    )
    return figure

if __name__ == '__main__':
    app.run_server(debug=True)